# I) More basic than basic of TensorFlow

To use tf lunch this notebook from docker.

Initialize docker.

Start the container

- `docker start -a coursera-aml-2`

To stop the container, in a different terminal window

- `docker stop -a coursera-aml-2`

Tf must be lunched after the code.

In [1]:
%config IPCompleter.greedy=True

In [2]:
import tensorflow as tf
import numpy as np
print(tf.__version__)

1.2.1


### 1) Let us initialize a graph for something as simple as multiplying and adding matrices

In [3]:
tf.reset_default_graph() 
# Because if one runs something (e.g. a placeholder) several time,
# use this reset to avoid graph to be cluttered

a = tf.placeholder(np.float32, (2, 2)) # For defining a tensor to be filled
b = tf.Variable(tf.ones((2, 2))) # Define a Variable
ct = tf.constant(np.ones((2,2)), dtype=tf.float32)

c = a @ b + ct # Multimplied both tensors and sum a constant tensor

In [4]:
print(c)

Tensor("add:0", shape=(2, 2), dtype=float32)


In [5]:
s = tf.InteractiveSession() 
# To run the graph we need a session object that encapsulates 
# the environment where operations are exectued and tensor objects 
# are evaluated.

In [6]:
s.run(tf.global_variables_initializer()) # compute initial values in graph execution environment 
s.run(c, feed_dict={a: np.ones((2, 2))}) # Feed the placeholder

array([[ 3.,  3.],
       [ 3.,  3.]], dtype=float32)

In [7]:
s.close() # Release resources for the session when not required

#### What is tf doing?

Every node in the graph is an operation.

In [8]:
tf.get_default_graph().get_operations()
# If we do not use tf.reset_default_graph(), it will create 
# a placeholder each time we run the cell and the graph will be 
# effectively cluttered.

[<tf.Operation 'Placeholder' type=Placeholder>,
 <tf.Operation 'ones' type=Const>,
 <tf.Operation 'Variable' type=VariableV2>,
 <tf.Operation 'Variable/Assign' type=Assign>,
 <tf.Operation 'Variable/read' type=Identity>,
 <tf.Operation 'Const' type=Const>,
 <tf.Operation 'matmul' type=MatMul>,
 <tf.Operation 'add' type=Add>,
 <tf.Operation 'init' type=NoOp>]

Let us see what tf is doing for each node.

The output tensor of each operation is given by:

In [9]:
print(tf.get_default_graph().get_operations()[0].outputs) # define placeholder
print(tf.get_default_graph().get_operations()[1].outputs) # define a tensor of 1's
print(tf.get_default_graph().get_operations()[2].outputs) # define tensor Variable
print(tf.get_default_graph().get_operations()[3].outputs) # assign Variable op.
print(tf.get_default_graph().get_operations()[4].outputs) # read Variable op.
print(tf.get_default_graph().get_operations()[5].outputs) # define constant ct
print(tf.get_default_graph().get_operations()[6].outputs) # multiplication op.
print(tf.get_default_graph().get_operations()[7].outputs) # addition
print(tf.get_default_graph().get_operations()[8].outputs) # init op.

[<tf.Tensor 'Placeholder:0' shape=(2, 2) dtype=float32>]
[<tf.Tensor 'ones:0' shape=(2, 2) dtype=float32>]
[<tf.Tensor 'Variable:0' shape=(2, 2) dtype=float32_ref>]
[<tf.Tensor 'Variable/Assign:0' shape=(2, 2) dtype=float32_ref>]
[<tf.Tensor 'Variable/read:0' shape=(2, 2) dtype=float32>]
[<tf.Tensor 'Const:0' shape=(2, 2) dtype=float32>]
[<tf.Tensor 'matmul:0' shape=(2, 2) dtype=float32>]
[<tf.Tensor 'add:0' shape=(2, 2) dtype=float32>]
[]


### 2) Let us just multiply two scalars as placeholders, feed them with data and see the operations in each node of the graph

In [10]:
tf.reset_default_graph()
# Create a placeholder data a
a = tf.placeholder(tf.float32, name="input1") 
# Create a placeholder data b
b = tf.placeholder(tf.float32, name="input2") 
# Create an multiply operation using a and b
c = tf.multiply(a, b, name="multiply_op")
s = tf.Session()
feed_dict = {a: 3.0, b: 2.0}
output = s.run(c, feed_dict)
print (output)
s.close()

6.0


### 3) Numbers can be multiplied just as constants

In [11]:
tf.reset_default_graph()
a = tf.constant(5.0)
b = tf.constant(6.0)
c = a * b
s = tf.InteractiveSession() # To run the graph
print(c)
print(s.run(c))

Tensor("mul:0", shape=(), dtype=float32)
30.0


In [12]:
s.run(c)
s.close()

# II) A first basic model in TensorFlow

### 4) Simple optimization (with prints in jupiyter)

In [13]:
tf.reset_default_graph()
x = tf.get_variable("x", shape=(), dtype=tf.float32#, trainable=True
                   )
# define scalar Variable x that can be updated during execution.
# All variables are trainable by default! If trainable=False variable 
# will not be updated during backpropagation
f = x ** 2

In [14]:
tf.get_default_graph().get_operations()

[<tf.Operation 'x/Initializer/random_uniform/shape' type=Const>,
 <tf.Operation 'x/Initializer/random_uniform/min' type=Const>,
 <tf.Operation 'x/Initializer/random_uniform/max' type=Const>,
 <tf.Operation 'x/Initializer/random_uniform/RandomUniform' type=RandomUniform>,
 <tf.Operation 'x/Initializer/random_uniform/sub' type=Sub>,
 <tf.Operation 'x/Initializer/random_uniform/mul' type=Mul>,
 <tf.Operation 'x/Initializer/random_uniform' type=Add>,
 <tf.Operation 'x' type=VariableV2>,
 <tf.Operation 'x/Assign' type=Assign>,
 <tf.Operation 'x/read' type=Identity>,
 <tf.Operation 'pow/y' type=Const>,
 <tf.Operation 'pow' type=Pow>]

Minimization of $f$ w.r.t. $x$

In [15]:
optimizer = tf.train.GradientDescentOptimizer(0.1) # 0.1 is the learning rate 
step = optimizer.minimize(f#, var_list=[x]
                         )
# minimize gives operation that can be run as many times as one wishes.
# As Variables are trainable by default, one has not to sprecify them:
# We can omit var_list=[x], but one can put trainable

In [16]:
tf.trainable_variables() # get trainable variables, i.g. x (scalar)

[<tf.Variable 'x:0' shape=() dtype=float32_ref>]

Let us make 10 gradient descent steps with a Python loop

In [17]:
with tf.Session() as s:  # in this way session will be closed automatically
    s.run(tf.global_variables_initializer())
    for i in range(10):
        _, curr_x, curr_f = s.run([step, x, f]) # "_": Ignore values when unpacking
        print(curr_x, curr_f)

-0.538244 0.452667
-0.430595 0.289707
-0.344476 0.185412
-0.275581 0.118664
-0.220465 0.075945
-0.176372 0.0486048
-0.141098 0.0311071
-0.112878 0.0199085
-0.0903024 0.0127414
-0.0722419 0.00815453


Both $f$ and $x$ goes to Zero, but the values are not synchronized because the order of opperations is not fixed.

### 5) Simple optimization (with tf.Print for synchronized output)

In [18]:
tf.reset_default_graph()
x = tf.get_variable("x", shape=(), dtype=tf.float32)
f = x ** 2
f = tf.Print(f, [x, f], "x, f:") # Prints are redirected to stdout when run

In [19]:
optimizer = tf.train.GradientDescentOptimizer(0.1)
step = optimizer.minimize(f)

In [20]:
with tf.Session() as s:
    s.run(tf.global_variables_initializer())
    for i in range(10):
        s.run([step, f])

Prints to jupyter server stdout the following synchronized output:

x, f:[0.69188243][0.74797076]

x, f:[0.69188243][0.47870129]

x, f:[0.55350596][0.30636886]

x, f:[0.44280475][0.19607605]

x, f:[0.35424381][0.12548868]

x, f:[0.28339505][0.080312759]

x, f:[0.22671604][0.051400162]

x, f:[0.18137284][0.032896105]

x, f:[0.14509827][0.021053508]

x, f:[0.11607862][0.013474245]

### 6) Simple optimization (with TensorBoard logging)

In [21]:
tf.reset_default_graph()
x = tf.get_variable("x", shape=(), dtype=tf.float32)
f = x ** 2

In [22]:
optimizer = tf.train.GradientDescentOptimizer(0.1)
step = optimizer.minimize(f)

In [23]:
tf.summary.scalar('curr_x', x)
tf.summary.scalar('curr_f', f)
summaries = tf.summary.merge_all() # Merge summaries in a single node.

In [24]:
s = tf.InteractiveSession()
summary_writer = tf.summary.FileWriter("logs/1", s.graph) 
# Run number at the end of the path. It will be useful for visualization
s.run(tf.global_variables_initializer())
for i in range(10):
    _, curr_summaries = s.run([step, summaries])
    summary_writer.add_summary(curr_summaries, i)
    summary_writer.flush()
# flush that data on disk so that you see that in a beautiful web interface. 
# This cell creates tf events in log folder.

At this point **TensorBoard** must be lunched

Execute:

In [30]:
!tensorboard --logdir=./logs

Starting TensorBoard b'54' at http://0496760811e8:6006
(Press CTRL+C to quit)
^C


Run  `tensorboard --logdir=./logs` in bash of Docker

This is displayed in browser
<img src="TensorBoard.jpg" style="width:70%">

In [25]:
s.close()

## III) Solve a linear regrassion

### 7) Training a linear model

Generate artificial data with liear structure but with some random noise:

In [26]:
# model data
N = 1000 # with 1000 exaples
D = 3    # and 3 features
x = np.random.random((N, D))
w = np.random.random((D, 1))
y = x @ w + np.random.randn(N, 1) * 0.20

print(x.shape, y.shape)
print(w.T) # traspose of w

(1000, 3) (1000, 1)
[[ 0.41150911  0.75097625  0.38692369]]


Operations for making **predictions**: 
- apply a linear model to all instances and store in  `predictions` tensor

- define a loss function to optimize parameters (e.g. Mean Squared Error)

In [27]:
tf.reset_default_graph()

# placeholders for the input data and the target
features = tf.placeholder(tf.float32, shape=(None, D))
target = tf.placeholder(tf.float32, shape=(None, 1))

weights = tf.get_variable("weights", shape=(D, 1), dtype=tf.float32)
predictions = features @ weights

loss = tf.reduce_mean((target - predictions) ** 2) 

print(features.shape, target.shape, predictions.shape, loss.shape)

(?, 3) (?, 1) (?, 1) ()


To run backpropagation an optimizer is needed (e.g. Gradient Descent Optimizer)

In [28]:
optimizer = tf.train.GradientDescentOptimizer(0.1)
step = optimizer.minimize(loss)

In [29]:
with tf.Session() as s:
    s.run(tf.global_variables_initializer())
    print('current loss every 50 iterations')
    for i in range(300):
        _, curr_loss, curr_weights = s.run([step, loss, weights], 
                                           feed_dict={features: x, target: y})
        if i % 50 == 0:
            print(curr_loss, curr_weights.T)

current loss every 50 iterations
0.361292 [[ 0.72192955  1.15506291  0.57392067]]
0.0385988 [[ 0.43164018  0.77441603  0.35024849]]
0.0381573 [[ 0.43643445  0.74086052  0.38003212]]
0.0380741 [[ 0.4386887   0.72623998  0.39288509]]
0.0380584 [[ 0.43973589  0.71986264  0.39842585]]
0.0380554 [[ 0.44022036  0.71708065  0.4008145 ]]


In [30]:
# found weights
curr_weights.T

array([[ 0.44044062,  0.71588272,  0.40183106]], dtype=float32)

In [31]:
# true weights
w.T

array([[ 0.41150911,  0.75097625,  0.38692369]])

In [32]:
s.close()

### 8) Model checkpoints

If training process takes a long time, it will benefict from model checkpoints. It can be saved the work, and continue if interrupted.

In [33]:
s = tf.InteractiveSession()
saver = tf.train.Saver(tf.trainable_variables())
s.run(tf.global_variables_initializer())
for i in range(300):
    _, curr_loss, curr_weights = s.run([step,loss, weights],
                                       feed_dict={features: x, target: y})
    if i % 50 == 0:
        saver.save(s,"logs/2/model.ckpt",global_step=i) 
        # Save trainable_variables in iterations to a desirable lkocation on disk (e.g. logs)
        print(curr_loss)

2.14964
0.0500833
0.0402815
0.038467
0.0381311
0.0380689


In [34]:
saver.last_checkpoints

['logs/2/model.ckpt-50',
 'logs/2/model.ckpt-100',
 'logs/2/model.ckpt-150',
 'logs/2/model.ckpt-200',
 'logs/2/model.ckpt-250']

Retrieve last saved checkpoints with restore method

It only contains tensor and vales, then only variables' values are restored. It does not contain graph definitions, therefore one needs to define a graph in the *same way* **before restoring** a checkpoint.

In [35]:
saver.restore(s,"logs/2/model.ckpt-50")

INFO:tensorflow:Restoring parameters from logs/2/model.ckpt-50


I won't see now how to continue iterating from there.

### Solve a linear regrassion (with TensorBoard logging) --Unfinished--

We use the same model data introduced before.

In [34]:
tf.reset_default_graph()

features = tf.placeholder(tf.float32, shape=(None, D))
target = tf.placeholder(tf.float32, shape=(None, 1))

weights = tf.get_variable("weights", shape=(D, 1), dtype=tf.float32)
predictions = features @ weights

loss = tf.reduce_mean((target - predictions) ** 2)

In [35]:
optimizer = tf.train.GradientDescentOptimizer(0.1)
step = optimizer.minimize(loss)

In [42]:
features

<tf.Tensor 'Placeholder:0' shape=(?, 3) dtype=float32>

In [38]:
tf.summary.scalar('curr_features', features[0])
tf.summary.scalar('curr_predictions', predictions)
summaries = tf.summary.merge_all() # Merge summaries in a single node.

In [39]:
with tf.Session() as s:
    summary_writer = tf.summary.FileWriter("logs/3", s.graph) # Where to store these logs. 
    s.run(tf.global_variables_initializer())
    print('current loss every 50 iterations')
    for i in range(300):
        _, curr_summaries, curr_loss, curr_weights = s.run(
            [step, summaries, loss, weights], 
            feed_dict={features: x, target: y})
        summary_writer.add_summary(curr_summaries, i)
        summary_writer.flush()
        if i % 50 == 0:
            print(curr_loss, curr_weights.T)

current loss every 50 iterations


InvalidArgumentError: tags and values not the same shape: [] != [1000,3] (tag 'c')
	 [[Node: c = ScalarSummary[T=DT_FLOAT, _device="/job:localhost/replica:0/task:0/cpu:0"](c/tags, _arg_Placeholder_0_0)]]

Caused by op 'c', defined at:
  File "/usr/lib/python3.5/runpy.py", line 184, in _run_module_as_main
    "__main__", mod_spec)
  File "/usr/lib/python3.5/runpy.py", line 85, in _run_code
    exec(code, run_globals)
  File "/usr/local/lib/python3.5/dist-packages/ipykernel_launcher.py", line 16, in <module>
    app.launch_new_instance()
  File "/usr/local/lib/python3.5/dist-packages/traitlets/config/application.py", line 658, in launch_instance
    app.start()
  File "/usr/local/lib/python3.5/dist-packages/ipykernel/kernelapp.py", line 486, in start
    self.io_loop.start()
  File "/usr/local/lib/python3.5/dist-packages/tornado/platform/asyncio.py", line 132, in start
    self.asyncio_loop.run_forever()
  File "/usr/lib/python3.5/asyncio/base_events.py", line 345, in run_forever
    sel